In [1]:
# Importing flask module in the project is mandatory 
# An object of Flask class is our WSGI application. 
import numpy as np
from flask import Flask, request, jsonify, render_template
import pickle
import json
import pandas as pd
#pd.set_option('precision', 15)

In [2]:
MAX_NB_WORDS = 50000
MAX_SEQUENCE_LENGTH = 250
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)

model = pickle.load(open('keras_model.pkl', 'rb'))

Using TensorFlow backend.
C:\Users\js198\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\js198\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\js198\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\js198\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [36]:
# Flask constructor takes the name of  
# current module (__name__) as argument. 
app = Flask(__name__) 

In [37]:
#Load the model which you created in your previous step

In [38]:
@app.route('/')
def home():
    return render_template('submit.html')

@app.route('/predict', methods=['POST'])
def predict():
    given_path = [x for x in request.form.values()]
    new_complaint = pd.read_csv(given_path[0])
    prediction_dept = []
    
    for complaint in pd.Series(new_complaint["Consumer complaint narrative"]):
        #print(complaint)
        seq = tokenizer.texts_to_sequences(complaint)
        padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
        model = pickle.load(open('keras_model.pkl', 'rb'))
        pred = model.predict(padded)
        labels = ['Credit reporting, credit repair services, or other personal consumer reports', 'Debt collection', 'Mortgage', 'Credit card or prepaid card', 'Student loan', 'Bank account or service', 'Checking or savings account', 'Consumer Loan', 'Payday loan, title loan, or personal loan', 'Vehicle loan or lease', 'Money transfer, virtual currency, or money service', 'Money transfers', 'Prepaid card']
        prediction_dept.append(labels[np.argmax(pred)])
        
    new_complaint['Predicted_Deparment'] =  prediction_dept
    new_complaint.to_csv(r'C:\Users\js198\Desktop\ML Training\Projects_and_Assignments\Project2_v1.0\output_file_newcomplaints.csv', index = False)
            
    return render_template('submit.html',prediction_text1="Prediction Process Completed. Output file created at path C:\\Users\\js198\\Desktop\\ML Training\\Projects_and_Assignments\\Project2_v1.0\\output_file_newcomplaints.csv")


In [ ]:
# main driver function 
if __name__ == '__main__': 
  
    # run() method of Flask class runs the application  
    # on the local development server. 
    app.run(port=8085)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8085/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Oct/2020 17:43:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Oct/2020 17:43:53] "POST /predict HTTP/1.1" 200 -


In [ ]:
# We need to click Kernel -> and then click Interrupt to come out of above program. This mean api will be stopped now afer interrupt.

In [ ]:
# the API created by this program is:-  "http://127.0.0.1:8085/FraudPredict/api"
# General testing:- When you run this on brouser, You need to get "Method Not Allowed" Then its working fine.
# When you call this api using Front-end app with Data, then predict function will be executed and predicted class will be returned